<a href="https://colab.research.google.com/github/stixmal/praktikum_project_ds/blob/main/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%80%D1%8B%D0%BD%D0%BE%D1%87%D0%BD%D0%BE%D0%B9%20%D1%81%D1%82%D0%BE%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D0%B8%20%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%BE%D0%B1%D0%B8%D0%BB%D1%8F/autos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Приложение для определения рыночной стоимости автомобиля

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Построим модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.  


Источник данных: `/datasets/autos.csv`.

## Оглавление

### [1. Подготовка данных](#1) <a id='10'></a>   



### [2. Обучение моделей](#2) <a id='20'></a>



### [3. Анализ скорости и качества моделей](#3) <a id='30'></a>


***

## 1. Подготовка данных <a id='1'></a>   [^](#10) 

In [ ]:
# импорт библиотеки pandas и numpy
import pandas as pd 
import numpy as np  

# импорт модуля display
from IPython.display import display  

# отключение предупреждений
import warnings
warnings.filterwarnings('ignore') 

# импорт модуля разделения на выборки
from sklearn.model_selection import train_test_split, GridSearchCV

# импорт скеллера
from sklearn.preprocessing import StandardScaler

# импорт модели линейной регрессии
from sklearn.linear_model import LinearRegression

# импорт модели случайного леса
from sklearn.ensemble import RandomForestRegressor

# импорт модели градиентного бустинга
import lightgbm as lgb

# импорт метрик
from sklearn.metrics import mean_squared_error, make_scorer

# импорт модуля времени
import time

In [ ]:
# чтение файла с данными
df = pd.read_csv('/datasets/autos.csv')

# визуальный осмотр  
df

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


Признаки:  

* **DateCrawled** — дата скачивания анкеты из базы
* **VehicleType** — тип автомобильного кузова
* **RegistrationYear** — год регистрации автомобиля
* **Gearbox** — тип коробки передач
* **Power** — мощность (л. с.)
* **Model** — модель автомобиля
* **Kilometer** — пробег (км)
* **RegistrationMonth** — месяц регистрации автомобиля
* **FuelType** — тип топлива
* **Brand** — марка автомобиля
* **NotRepaired** — была машина в ремонте или нет
* **DateCreated** — дата создания анкеты
* **NumberOfPictures** — количество фотографий автомобиля
* **PostalCode** — почтовый индекс владельца анкеты (пользователя)
* **LastSeen** — дата последней активности пользователя  

Целевой признак:  

* **Price** - цена в евро.

Удалили признаки, невлияющие на предсказания модели:  

* дата скачивания анкеты из базы,  
* дата создания анкеты, 
* количество фотографий автомобиля,  
* почтовый индекс владельца анкеты,  
* дата последней активности пользователя.

In [ ]:
# удаление неактуальных признаков
df = df.drop(df[['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']], axis=1)
df

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [ ]:
# информация о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [ ]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645
std,4514.158514,90.227958,189.850405,37905.341530,3.726421
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


Видим, что в столбцах имеются пропущенные значения и выбросы в признаках цены, мощности автомобиля и года регистрации.

In [ ]:
# избавление от выбросов
df_new = df.query('Price > 100 and ( 1900 <= RegistrationYear <= 2020) and Power < 1000')

# очистка таблицы составила не более 5 %
len(df_new) / len(df)

0.9584190490703194

In [ ]:
# провели прямое кодирование категориальных признаков с избеганием дамми-ловушки 
data_ohe = pd.get_dummies(df_new, drop_first=True)

# переменные с признаками и ответами
X = data_ohe.drop(['Price'], axis=1)
y = data_ohe['Price']

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=12345)

Проконтролировали размер выборок.

In [ ]:
X_train.shape, y_train.shape

((254725, 307), (254725,))

In [ ]:
X_test.shape, y_test.shape

((84909, 307), (84909,))

In [ ]:
data_ohe.shape

(339634, 308)

In [ ]:
# провели масштабирование числовых признаков
numeric = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

In [ ]:
# cоздали объект структуры StandardScaler() и настроили его на обучающих данных
scaler = StandardScaler()
scaler.fit(X_train[numeric])

# преобразовали обучающую и тестовую выборки функцией transform()
X_train[numeric] = scaler.transform(X_train[numeric])
X_test[numeric] = scaler.transform(X_test[numeric])

## 2. Обучение моделей <a id='2'></a>  [^](#10) 

* ### 2.1 Линейная регрессия

In [ ]:
lin_reg = LinearRegression(n_jobs=-1)

In [ ]:
# время обучения модели
start = time.time()
lin_reg.fit(X_train, y_train)
end_fit = time.time() - start
end_fit

28.85953426361084

In [ ]:
# параметры модели
lin_reg.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'normalize': False}

In [ ]:
# время предсказания модели
start = time.time()
y_pred = lin_reg.predict(X_test)
end_pred = time.time() - start
end_pred

0.1830449104309082

In [ ]:
# score и rmse модели
score = lin_reg.score(X_test, y_test)
score

0.6266601656842947

In [ ]:
rmse = mean_squared_error(y_test, y_pred) ** 0.5
rmse

2756.6020625766455

In [ ]:
# результаты сохраним в results
results = []
results.append(('Lin_Reg', end_fit, end_pred, score, rmse))

* ### 2.2 Случайный лес

In [ ]:
rfr = RandomForestRegressor(n_jobs=-1, random_state=12345)

In [ ]:
# время обучения модели
start = time.time()
rfr.fit(X_train, y_train)
end_fit = time.time() - start
end_fit

80.56161952018738

In [ ]:
# параметры модели
rfr.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 12345,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# время предсказания модели
start = time.time()
y_pred = rfr.predict(X_test)
end_pred = time.time() - start
end_pred

0.7742817401885986

In [ ]:
# score и rmse модели
score = rfr.score(X_test, y_test)

In [ ]:
rmse = mean_squared_error(y_test, y_pred) ** 0.5
rmse

1683.4602613053873

In [ ]:
# результаты сохраним в results
results.append(('Random_Forest', end_fit, end_pred, score, rmse))

* ### 2.3 Градиентный бустинг LightGBM

Так как LightGBM поддерживает категориальные переменные, преобразовали их в `int`.

In [ ]:
# переменные с признаками и ответами
X = df_new.drop(['Price'], axis=1)
y = df_new['Price']

In [ ]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [ ]:
# кодирование категориальных признаков
for i in cat_features:
    X[i] = X[i].astype('category').cat.codes

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=12345)

Проконтролировали размер выборок.

In [ ]:
X_train.shape, y_train.shape

((254725, 10), (254725,))

In [ ]:
X_test.shape, y_test.shape

((84909, 10), (84909,))

In [ ]:
df_new.shape

(339634, 11)

In [ ]:
# функция нахождения лучшей модели по поиску гиперпараметров GridSearchCV
def grid_search(model, param_grid, cv, X_train, y_train):
    grid_model = GridSearchCV(model, param_grid=param_grid, cv=cv, n_jobs=-1)
    grid_model.fit(X_train, y_train)
    best_estimator = grid_model.best_estimator_
    best_parameters = grid_model.best_params_
    best_score = grid_model.best_score_
    return best_estimator, best_parameters, best_score

In [ ]:
# сетка гиперпараметров градиентного бустинга
param_grid={'learning_rate': [0.01, 0.1, 1],
            'n_estimators': [20, 40],
           }

In [ ]:
# вызов LightGBM
gbm = lgb.LGBMRegressor(num_leaves=31, random_state=12345)

In [ ]:
# нахожение лучшей модели по сетке гиперпараметров
grid_bust = grid_search(gbm, param_grid, 3, X_train, y_train)
grid_bust

(LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=40, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 {'learning_rate': 1, 'n_estimators': 40},
 0.8389217369065698)

In [ ]:
gbm = lgb.LGBMRegressor(num_leaves=31, learning_rate=grid_bust[1]['learning_rate'],
                        n_estimators=grid_bust[1]['n_estimators'], random_state=12345)

In [ ]:
# время обучения лучшей модели
start = time.time()
gbm.fit(X_train, y_train)
end_fit = time.time() - start
end_fit

4.3850438594818115

In [ ]:
# время предсказания модели
start = time.time()
y_pred = gbm.predict(X_test)
end_pred = time.time() - start
end_pred

0.2973175048828125

In [ ]:
# rmse модели
rmse = mean_squared_error(y_test, y_pred) ** 0.5
rmse

1794.9439418299862

In [ ]:
# результаты сохраним в results
results.append(('LightGBM', end_fit, end_pred, grid_bust[2], rmse))

## 3. Анализ скорости и качества моделей <a id='3'></a>  [^](#10) 

Свели результаты по всем моделям.

In [ ]:
results_all = pd.DataFrame(results, columns=['Модель', 'Время обучения, c', 'Время предсказания, c', 'Score', 'RMSE'])
results_all

,Модель,"Время обучения, c","Время предсказания, c",Score,RMSE
0,Lin_Reg,28.859534,0.183045,0.626660,2756.602063
1,Random_Forest,80.561620,0.774282,0.860761,1683.460261
2,LightGBM,4.385044,0.297318,0.838922,1794.943942


## Вывод:  

* Качество предсказаний линейной регрессии значительно проигрывает двум другим моделям.  
* Время обучения случайного леса даже без учета перебора гиперпараметров превышает 1 минуту. Однако и качество предсказаний также существенно выше, чем у линейной регрессии.
* Качество градиентного бустинга LightGBM сопоставимо с качеством случайного леса. Время обучения различается довольно сильно, что несомненно идёт в плюс бустингу.
* Все модели предсказывают довольно быстро - в пределах одной секунды.
* Модель градиентного бустинга является предпочтительней в виду высокой скорости обучения и качества предсказаний.